本教程提供了如何将 pandas dataframes 加载到 tf.data.Dataset。   

本教程使用了一个小型数据集，由克利夫兰诊所心脏病基金会（Cleveland Clinic Foundation for Heart Disease）提供. 此数据集中有几百行CSV。每行表示一个患者，每列表示一个属性（describe）。我们将使用这些信息来预测患者是否患有心脏病，这是一个二分类问题。

### 使用 pandas 读取数据

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
import tensorflow as tf

In [2]:
# 下载包含心脏数据集的 csv 文件。
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv') # 文件名称  原始下载地址 return：存储路径

In [4]:
# 使用 pandas读取csv文件
df = pd.read_csv(csv_file)

'C:\\Users\\admin\\.keras\\datasets\\heart.csv'

In [6]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [7]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

In [8]:
# 将thal列转换为离散数值
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes

In [9]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,2,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,4,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0


### 使用 tf.data.Dataset 读取数据
使用 tf.data.Dataset.from_tensor_slices 从 pandas dataframe 中读取数值。  

使用 tf.data.Dataset 的其中一个优势是可以允许您写一些简单而又高效的数据管道（data pipelines)。从 loading data guide 可以了解更多。

In [11]:
target = df.pop('target')

In [12]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [13]:
for feat, targ in dataset.take(5):
    print('Featrues:{}, Target:{}'.format(feat, targ))

Featrues:[ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   2. ], Target:0
Featrues:[ 67.    1.    4.  160.  286.    0.    2.  108.    1.    1.5   2.    3.
   3. ], Target:1
Featrues:[ 67.    1.    4.  120.  229.    0.    2.  129.    1.    2.6   2.    2.
   4. ], Target:0
Featrues:[ 37.    1.    3.  130.  250.    0.    0.  187.    0.    3.5   3.    0.
   3. ], Target:0
Featrues:[ 41.    0.    2.  130.  204.    0.    2.  172.    0.    1.4   1.    0.
   3. ], Target:0


In [15]:
dataset

<TensorSliceDataset shapes: ((13,), ()), types: (tf.float64, tf.int64)>

In [16]:
# 由于 pd.Series 实现了 __array__ 协议，因此几乎可以在任何使用 np.array 或 tf.Tensor 的地方透明地使用它。
tf.constant(df['thal'])

<tf.Tensor: id=21, shape=(303,), dtype=int32, numpy=
array([2, 3, 4, 3, 3, 3, 3, 3, 4, 4, 2, 3, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3,
       3, 4, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 4, 2, 4, 3, 4, 3, 4, 4,
       2, 3, 3, 4, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 4,
       4, 2, 3, 3, 4, 3, 4, 3, 3, 4, 4, 3, 3, 4, 4, 3, 3, 3, 3, 4, 4, 4,
       3, 3, 4, 3, 4, 4, 3, 4, 3, 3, 3, 4, 3, 4, 4, 3, 3, 4, 4, 4, 4, 4,
       3, 3, 3, 3, 4, 3, 4, 3, 4, 4, 3, 3, 2, 4, 4, 2, 3, 3, 4, 4, 3, 4,
       3, 3, 4, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4,
       4, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 2,
       4, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 2, 2, 4, 3, 4, 2, 4, 3,
       3, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 2, 2, 4, 3, 4, 3, 2, 4, 3, 3, 2,
       4, 4, 4, 4, 3, 0, 3, 3, 3, 3, 1, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 4,
       3, 3, 4, 4, 4, 4, 3, 3, 4, 3, 4, 3, 4, 4, 3, 4, 4, 3, 4, 4, 3, 3

In [17]:
# 随机读取（shuffle）并批量处理数据集。
train_dataset = dataset.shuffle(len(df)).batch(1)

### 创建模型并训练

In [21]:
def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
  ])

    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
    return model

In [22]:
model = get_compiled_model()
model.fit(train_dataset, epochs=5)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/5
303/303 [==============================] - 2s 7ms/step - loss: 1.6589 - accuracy: 0.5842
Epoch 2/5
303/303 [==============================] - 0s 1ms/step - loss: 0.7450 - accuracy: 0.6832
Epoch 3/5
303/303 [==============================] - 0s 1ms/step - loss: 0.6500 - accuracy: 0.7096
Epoch 4/5
303/303 [==============================] - 0s 1ms/step - loss: 0.6287 - accuracy: 0.6997
Epoch 5/5
303/303 [==============================] - 0s 1ms/step - loss: 0.5542 - accuracy: 0.7492


### 使用字典代替特征列
将字典作为输入传输给模型就像创建 tf.keras.layers.Input 层的匹配字典一样简单，应用任何预处理并使用 functional api。 您可以使用它作为 feature columns 的替代方法。

In [23]:
inputs = {key:tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}


In [33]:
x = tf.stack(list(inputs.values()), axis=-1)
x.shape

TensorShape([None, 13])

In [35]:
x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)
model_func.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

与 tf.data 一起使用时，保存 pd.DataFrame 列结构的最简单方法是将 pd.DataFrame 转换为 dict ，并对该字典进行切片。

In [37]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)

In [38]:
for dict_slice in dict_slices.take(1):
    print(dict_slice)

({'age': <tf.Tensor: id=5688, shape=(16,), dtype=int32, numpy=array([63, 67, 67, 37, 41, 56, 62, 57, 63, 53, 57, 56, 56, 44, 52, 57])>, 'sex': <tf.Tensor: id=5696, shape=(16,), dtype=int32, numpy=array([1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1])>, 'cp': <tf.Tensor: id=5691, shape=(16,), dtype=int32, numpy=array([1, 4, 4, 3, 2, 2, 4, 4, 4, 4, 4, 2, 3, 2, 3, 3])>, 'trestbps': <tf.Tensor: id=5700, shape=(16,), dtype=int32, numpy=
array([145, 160, 120, 130, 130, 120, 140, 120, 130, 140, 140, 140, 130,
       120, 172, 150])>, 'chol': <tf.Tensor: id=5690, shape=(16,), dtype=int32, numpy=
array([233, 286, 229, 250, 204, 236, 268, 354, 254, 203, 192, 294, 256,
       263, 199, 168])>, 'fbs': <tf.Tensor: id=5693, shape=(16,), dtype=int32, numpy=array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0])>, 'restecg': <tf.Tensor: id=5695, shape=(16,), dtype=int32, numpy=array([2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0])>, 'thalach': <tf.Tensor: id=5699, shape=(16,), dtype=int32, numpy=
ar

In [39]:
model_func.fit(dict_slices, epochs=15)

Epoch 1/15
19/19 [==============================] - 1s 28ms/step - loss: 1.8750 - accuracy: 0.5281
Epoch 2/15
19/19 [==============================] - 0s 5ms/step - loss: 1.0090 - accuracy: 0.6535
Epoch 3/15
19/19 [==============================] - 0s 5ms/step - loss: 0.7590 - accuracy: 0.6898
Epoch 4/15
19/19 [==============================] - 0s 5ms/step - loss: 0.6914 - accuracy: 0.6964
Epoch 5/15
19/19 [==============================] - 0s 5ms/step - loss: 0.6450 - accuracy: 0.6898
Epoch 6/15
19/19 [==============================] - 0s 5ms/step - loss: 0.6149 - accuracy: 0.7063
Epoch 7/15
19/19 [==============================] - 0s 5ms/step - loss: 0.5948 - accuracy: 0.7162
Epoch 8/15
19/19 [==============================] - 0s 5ms/step - loss: 0.5805 - accuracy: 0.7261
Epoch 9/15
19/19 [==============================] - 0s 5ms/step - loss: 0.5634 - accuracy: 0.7261
Epoch 10/15
19/19 [==============================] - 0s 5ms/step - loss: 0.5447 - accuracy: 0.7327
Epoch 11/15
19/19 